In [1]:
import pandas as pd
import numpy as np
import os
from importlib import reload

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

from sklearn.model_selection import train_test_split
import skimage
from skimage import io
from skimage.transform import resize

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy

# local imports
import model as _model
import utils as _tools

In [2]:
"""
control the generation of trainning and dev datasets
"""

train, dev, _ = _tools.make_trainning_data(sample=10000, 
                                           return_frames=True, 
                                           state=1729)

print()
print("Trainning label distribution")
print(train['Cardiomegaly'].value_counts(normalize=True, dropna=False))

print()
print("Development label distribution")
print(dev['Cardiomegaly'].value_counts(normalize=True, dropna=False))

sampling 10000 records
train.shape: (7500, 19)
dev.shape: (2500, 19)
valid.shape: (234, 19)
saved: ./train.csv
saved: ./dev.csv
saved: ./valid.csv

Trainning label distribution
 NaN    0.790000
 1.0    0.125067
 0.0    0.049600
-1.0    0.035333
Name: Cardiomegaly, dtype: float64

Development label distribution
 NaN    0.8052
 1.0    0.1148
 0.0    0.0480
-1.0    0.0320
Name: Cardiomegaly, dtype: float64


In [3]:
"""
build and train the model
"""

reload(_model)

# build the models
resnet = _model.TransferModel()

# train the model
resnet.train()


-------------------------------
Cardiomegaly Model epoch 1/30


/sw/arcts/centos7/python3.8-anaconda/2020.07/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Trainning loss: 0.6965 accuracy: 32.53 %
Validation loss: 0.7004 accuracy: 33.44 %

-------------------------------
Cardiomegaly Model epoch 2/30
Trainning loss: 0.6661 accuracy: 42.41 %
Validation loss: 0.6887 accuracy: 39.52 %

-------------------------------
Cardiomegaly Model epoch 3/30
Trainning loss: 0.6477 accuracy: 47.47 %
Validation loss: 0.6754 accuracy: 49.40 %

-------------------------------
Cardiomegaly Model epoch 4/30
Trainning loss: 0.6329 accuracy: 50.15 %
Validation loss: 0.6690 accuracy: 48.16 %

-------------------------------
Cardiomegaly Model epoch 5/30
Trainning loss: 0.6204 accuracy: 55.16 %
Validation loss: 0.6621 accuracy: 54.56 %

-------------------------------
Cardiomegaly Model epoch 6/30
Trainning loss: 0.6011 accuracy: 56.23 %
Validation loss: 0.6556 accuracy: 53.16 %

-------------------------------
Cardiomegaly Model epoch 7/30
Trainning loss: 0.5924 accuracy: 57.33 %
Validation loss: 0.6496 accuracy: 55.24 %

-------------------------------
Cardiome

In [ ]:
# get results on dev set
results = resnet.evaluate_model(resnet.best_model, 
                                resnet.dataloader_dev, 
                                resnet.dev_map)
print(results.shape)

# get distributions of true labels
print()
print(results['y_true'].value_counts(normalize=True))

# get distributions of pred labels
print()
print(results['y_pred'].value_counts(normalize=True))

In [ ]:
results.head()

In [ ]:
matplotlib.rcParams['figure.dpi'] = 150
results['y_prob'].hist(edgecolor='black', bins=30)
plt.title('Distribution of Propensities')

In [ ]:
_time = list(range(len(resnet.train_loss_history)))

matplotlib.rcParams['figure.dpi'] = 150
plt.plot(_time, resnet.train_loss_history, c='red', label="Trainning")
plt.plot(_time, resnet.dev_loss_history, c='green', label="Testing")
plt.title("Trainning Loss")
plt.xlabel("Epoch")
plt.ylabel("Cross Entropy Loss")
plt.legend()

In [ ]:
_time = list(range(len(resnet.train_acc_history)))

matplotlib.rcParams['figure.dpi'] = 150
plt.plot(_time, resnet.train_acc_history, c='red', label="Trainning")
plt.plot(_time, resnet.dev_acc_history, c='green', label="Testing")
plt.title("Trainning Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

In [ ]:
reload(_tools)

res = _tools.get_classification_metrics(results)
res

In [ ]:
"""
save the model
"""

outpath = "models/resnet18.pth"
torch.save(resnet.model.state_dict(), outpath)
print(f"saved: {outpath}")

In [ ]:
# get results on valid set
results = resnet.evaluate_model(resnet.best_model, 
                                resnet.dataloader_valid, 
                                resnet.valid_map)
print(results.shape)

# get distributions of true labels
print()
print(results['y_true'].value_counts(normalize=True))

# get distributions of pred labels
print()
print(results['y_pred'].value_counts(normalize=True))

print()
res = _tools.get_classification_metrics(results)
res

In [ ]:
!git add .

In [ ]:
!git commit -m "model updates"

In [ ]:
!git push